In [ ]:
%pip install langchain langchain-ollama neo4j sentence-transformers llama-index
%pip install pandas numpy
%pip install requests
%pip install python-dotenv

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from neo4j import GraphDatabase
from sentence_transformers import SentenceTransformer

from langchain import LLMChain, PromptTemplate
import requests

from dotenv import load_dotenv
import os

In [12]:
# Custom API: using deepbricks API

# Configure Deepbricks API and model
BASE_URL = ""
API_KEY = ""
MODEL_NAME = ""
temperature = 0.7
max_tokens = 150

class CustomAPI:
    def __init__(self, base_url, api_key, model):
        self.base_url = base_url
        self.headers = {
            'Authorization': f'Bearer {api_key}',
            'Content-Type': 'application/json',
        }
        self.model = model

    def generate(self,messages, tem=temperature, max_t=max_tokens):
        payload = {
            'model': self.model,  # add model name
            'messages': messages,  # send messages as context
            'temperature': tem, # add temperature
            'max_tokens': max_t # add max tokens
        }

        response = requests.post(
            f'{self.base_url}',
            headers=self.headers,
            json=payload
        )

        if response.status_code == 200:
            return response.json()['choices'][0]['message']['content'].strip()
        else:
            raise Exception(f"API call failed: {response.status_code}, {response.text}")

# using Custom API class
llm = CustomAPI(base_url=BASE_URL, api_key=API_KEY, model=MODEL_NAME)

In [4]:
# read the CSV file
data = pd.read_csv('test-data-2.csv')

# Replace NaN values with empty strings for Neo4j compatibility
data = data.replace({np.nan: ''})

# Connect to Neo4j
driver = GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j", "test_password"))

# Creating Garment Nodes and Relationships Between Them, Creating Graph Nodes and Relationships in Neo4j
def create_clothing_graph(tx, row):
    query = (
        """
        MERGE (c:Clothing {id: $id})
        SET c += {name: $name, brand: $brand, type: $type, group: $group, details: $details, 
                price: $price, currency: $currency, color: $color, size: $size, 
                style: $style, pattern: $pattern, material: $material, occasion: $occasion}
        MERGE (b:Brand {name: $brand})
        MERGE (t:Type {name: $type})
        MERGE (b)-[:MAKES]->(c)
        MERGE (c)-[:BELONGS_TO]->(t)
        """
    )
    tx.run(query, id=row['id'], name=row['name'], brand=row['brand'], type=row['type'], group=row['group'], 
           details=row['details'], price=row['price'], currency=row['currency'], color=row['color'], 
           size=row['size'], style=row['style'], pattern=row['pattern'], material=row['material'], 
           occasion=row['occasion'])

# Importing data into Neo4j and building graphs
with driver.session() as session:
    data.apply(lambda row: session.execute_write(create_clothing_graph, row), axis=1)

# Deleting full-text indexes
def delete_fulltext_index():
    with driver.session() as session:
        session.run("DROP INDEX clothingIndex IF EXISTS")

# Creating Full-Text Indexes
def create_fulltext_index():
    with driver.session() as session:
        session.run("""
        CREATE FULLTEXT INDEX clothingIndex
        FOR (c:Clothing)
        ON EACH [c.id, c.name, c.brand, c.details]
        """)

# Calling the Delete Index and Create Index functions after a garment node has been created
delete_fulltext_index()  # Delete the index first (if it exists)
create_fulltext_index()  # Then create a new index

In [ ]:
# Loading Embedded Models
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate a description embedding vector for each garment
data['description'] = data[['name', 'brand', 'type', 'details']].apply(lambda x: ' '.join(x), axis=1)
embeddings = model.encode(data['description'].tolist())

# Storage embedded in Neo4j
def store_embedding(tx, name, embedding):
    query = "MATCH (c:Clothing {name: $name}) SET c.embedding = $embedding"
    tx.run(query, name=name, embedding=embedding.tolist())

with driver.session() as session:
    for i, row in data.iterrows():
        session.write_transaction(store_embedding, row['name'], embeddings[i])

In [7]:
# Define a function for full-text search
def retrieve_products_fulltext(query):
    with driver.session() as session:
        # Finding matches in Clothing nodes using full-text indexing
        result = session.run("""
        CALL db.index.fulltext.queryNodes('clothingIndex', $query)
        YIELD node, score
        RETURN node.id as id, node.name AS name, node.brand AS brand, node.details AS details, score
        ORDER BY score DESC LIMIT 10
        """, {"query": query})
        
        # Prints the results of each match
        products = []
        for record in result:
            
            print(f"ID: {record['id']}, Name: {record['name']}, Brand: {record['brand']}, Details: {record['details']}, Score: {record['score']}")
            
            products.append({
                "id": record["id"],  # Include ID
                "name": record["name"],
                "brand": record["brand"],
                "details": record["details"],
                "score": record["score"]
            })

    return products

In [8]:
# Defining a prompt template
template = """
User is searching for clothing. The input query is: "{input_query}".
The following product data matches the query: 
{product_data}.
Generate a response to suggest clothing.
"""
prompt = PromptTemplate(input_variables=["input_query", "product_data"], template=template)

# Defining the Generate Response Function
def generate_response(openai_model, input_query, product_data):
    combined_prompt = """
User is searching for clothing. The input query is: "{}".
The following product data matches the query:
{}.
Generate a response to suggest clothing.
""".format(input_query, product_data)

    messages = [
        {"role": "system", "content": "You are a helpful fashion assistant"},
        {"role": "user", "content": combined_prompt}
    ]

    # Calling the OpenAI API to generate a response
    response = openai_model.generate(messages) 
    
    return response

In [9]:
# topic Keywords

# TODO: extract the keywords using NLP Lib: SpaCy

# TODO: Add more keywords to improve topic detection accuracy
on_topic_keywords = ['jeans', 'dress', 'shirt', 'pants', 'clothing', 'size', 'color', 'fashion', 'outfit', 'trousers', 'jacket']
# TODO: 
off_topic_keywords = ['weather', 'news', 'movie', 'sports', 'politics', 'celebrity', 'tv show', 'music', 'event']

def detect_topic(user_input):
    if any(word in user_input.lower() for word in on_topic_keywords):
        return "on_topic"
    elif any(word in user_input.lower() for word in off_topic_keywords):
        return "off_topic"
    else:
        return "neutral"

In [ ]:
def search_and_generate_response(user_query):
    # Retrieving Matching Clothing from Neo4j Using Full-Text Indexing
    similar_products = retrieve_products_fulltext(user_query)  # Full-text search using user queries
    print("=====================================================")
    print(f"Similar products: {similar_products}")

    # Formatting product data for prompt entry
    product_data = ", ".join([f"{p['name']} (ID: {p['id']}, Brand: {p['brand']}, Details: {p['details']})" for p in similar_products])
    
    # Print formatted product data for commissioning
    print("=====================================================")
    print(f"Formatted product data: {product_data}")

    # Generating Responses with Custom OpenAI Classes
    response = generate_response(llm, user_query, product_data)
    
    # Returns the generated response and product data (in JSON format)
    return {
        "response": response,
        "products": similar_products  # Return to full product information
    }

# Sample usage
user_input = "I'm looking for a white shirt with brand huili"

if detect_topic(user_input) == "on_topic":
    result = search_and_generate_response(user_input)
    print("=====================================================")
    print(result)
elif detect_topic(user_input) == "off_topic":
    print({"response": "It seems your question is off-topic. Do you want to search for clothing?"})